Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices
from tqdm import tqdm
import numpy

import lab9_lib
from GA import Ga, Individual

Toy example to show how it works

In [2]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    # ind = [1] * 50
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11000000110010110110000101000010101011111010100001: 29.56%
00101100110010101000011001111110100010001000111111: 15.33%
11001010100010010111000110000000001010111001110111: 15.36%
10011111111000100101111101111000111101111011101011: 47.33%
10110111100000001111101010110100110011110110101010: 9.11%
11101110000100111111110110000100011011001110101001: 9.11%
11011111110000111101001111001000001011100010010011: 23.33%
11000011111110001001011010011000010100100100011101: 7.33%
10000010111101101100010010101001100011000000010101: 7.34%
00010101111110101001101001100101001101101001000010: 23.56%
10


## Now we try to solve the problem using EAs or similar and minimize the number of fitness calls

In [43]:
sizes = [1, 2, 5, 10]
k = 1000

POPULATION_SIZE = 10
OFFSPRING_SIZE = 50
N = 10
PM = 0.1
TOURNAMENT_SIZE = 2

for problem_size in sizes:
    fitness = lab9_lib.make_problem(problem_size)

    population = Ga(POPULATION_SIZE, fitness, N)
    for _ in range(1000):
        #solve the problem
        population.generate_offspring(OFFSPRING_SIZE, PM, TOURNAMENT_SIZE)
        population.survival_selection(POPULATION_SIZE)

    print(f"{problem_size} -> {population.population[0].fitness:.2%}, used {fitness.calls} call(s)\n")

1 -> 85.40%, used 50010 call(s)

2 -> 74.80%, used 50010 call(s)

5 -> 39.43%, used 50010 call(s)

10 -> 23.55%, used 50010 call(s)



In [4]:
i = Individual(numpy.array([1,2,3,4,5,6]), fitness)

numpy.concatenate((i.genome[:2], i.genome[3:]))

array([1, 2, 4, 5, 6])

In [38]:
POPULATION_SIZE = 10
OFFSPRING_SIZE = 50
N = 10
PM = 0.1
TOURNAMENT_SIZE = 2

population = Ga(POPULATION_SIZE, fitness, N)
population.generate_offspring(OFFSPRING_SIZE, PM, TOURNAMENT_SIZE)
population.survival_selection(POPULATION_SIZE)
population.population[0].fitness

0.11600455833

In [39]:
fitness.calls

1171